# 두개의 데이터에 대해서 한번에 해보기

In [1]:
# 1. Import Module
import os, sys
import random
#import itertools
#import io
#import math
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2
import pandas as pd
import math
import time

from tqdm import tqdm

from torchvision import datasets, transforms
from transformers import GLPNForDepthEstimation, GLPNFeatureExtractor
from PIL import Image

In [15]:
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# Preprocessing and bounding box
# standard PyTorch mean-std input image normalization
dpt_transform = transforms.Compose([
    #transforms.Resize(800),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

# Detection
def detect(im, model, transform):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0)
    #img_shape = img.shape

    # demo model only support by default images with aspect ratio between 0.5 and 2
    # if you want to use images with an aspect ratio outside this range
    # rescale your image so that the maximum size is at most 1333 for best results
    assert img.shape[-2] <= 1600 and img.shape[-1] <= 1600, 'demo model only supports images up to 1600 pixels on each side'

    # propagate through the model
    outputs = model(img)

    # keep only predictions with 0.7+ confidence
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.7

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
    return probas[keep], bboxes_scaled

detr_101 = torch.hub.load('facebookresearch/detr:main', 'detr_resnet101', pretrained=True) # Use resnet 101
detr_101.eval()
feature_extractor = GLPNFeatureExtractor.from_pretrained("vinvino02/glpn-kitti") # glpn-nyu2
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-kitti")
model.eval()

Using cache found in C:\Users\Admin/.cache\torch\hub\facebookresearch_detr_main


GLPNForDepthEstimation(
  (glpn): GLPNModel(
    (encoder): GLPNEncoder(
      (patch_embeddings): ModuleList(
        (0): GLPNOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): GLPNOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): GLPNOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): GLPNOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
      )
      (block): ModuleList(

In [16]:
# scene1
scene1_clone = os.listdir('./VKITTI/Scene01/clone/frames/rgb/Camera_1/')
scene1_clone_bbox = pd.read_csv('./VKITTI_txt/Scene01/clone/bbox.txt', delimiter=' ')
scene1_clone_info = pd.read_csv('./VKITTI_txt/Scene01/clone/info.txt', delimiter=' ')
scene1_clone_pose = pd.read_csv('./VKITTI_txt/Scene01/clone/pose.txt', delimiter=' ')

scene1_fog = os.listdir('./VKITTI/Scene01/fog/frames/rgb/Camera_1/')
scene1_fog_bbox = pd.read_csv('./VKITTI_txt/Scene01/fog/bbox.txt', delimiter=' ')
scene1_fog_info = pd.read_csv('./VKITTI_txt/Scene01/fog/info.txt', delimiter=' ')
scene1_fog_pose = pd.read_csv('./VKITTI_txt/Scene01/fog/pose.txt', delimiter=' ')

scene1_morning = os.listdir('./VKITTI/Scene01/morning/frames/rgb/Camera_1/')
scene1_morning_bbox = pd.read_csv('./VKITTI_txt/Scene01/morning/bbox.txt', delimiter=' ')
scene1_morning_info = pd.read_csv('./VKITTI_txt/Scene01/morning/info.txt', delimiter=' ')
scene1_morning_pose = pd.read_csv('./VKITTI_txt/Scene01/morning/pose.txt', delimiter=' ')

scene1_overcast = os.listdir('./VKITTI/Scene01/overcast/frames/rgb/Camera_1/')
scene1_overcast_bbox = pd.read_csv('./VKITTI_txt/Scene01/overcast/bbox.txt', delimiter=' ')
scene1_overcast_info = pd.read_csv('./VKITTI_txt/Scene01/overcast/info.txt', delimiter=' ')
scene1_overcast_pose = pd.read_csv('./VKITTI_txt/Scene01/overcast/pose.txt', delimiter=' ')

scene1_rain = os.listdir('./VKITTI/Scene01/rain/frames/rgb/Camera_1/')
scene1_rain_bbox = pd.read_csv('./VKITTI_txt/Scene01/rain/bbox.txt', delimiter=' ')
scene1_rain_info = pd.read_csv('./VKITTI_txt/Scene01/rain/info.txt', delimiter=' ')
scene1_rain_pose = pd.read_csv('./VKITTI_txt/Scene01/rain/pose.txt', delimiter=' ')

scene1_sunset = os.listdir('./VKITTI/Scene01/sunset/frames/rgb/Camera_1/')
scene1_sunset_bbox = pd.read_csv('./VKITTI_txt/Scene01/sunset/bbox.txt', delimiter=' ')
scene1_sunset_info = pd.read_csv('./VKITTI_txt/Scene01/sunset/info.txt', delimiter=' ')
scene1_sunset_pose = pd.read_csv('./VKITTI_txt/Scene01/sunset/pose.txt', delimiter=' ')

# scene2
scene2_clone = os.listdir('./VKITTI/Scene02/clone/frames/rgb/Camera_1/')
scene2_clone_bbox = pd.read_csv('./VKITTI_txt/Scene02/clone/bbox.txt', delimiter=' ')
scene2_clone_info = pd.read_csv('./VKITTI_txt/Scene02/clone/info.txt', delimiter=' ')
scene2_clone_pose = pd.read_csv('./VKITTI_txt/Scene02/clone/pose.txt', delimiter=' ')

scene2_fog = os.listdir('./VKITTI/Scene02/fog/frames/rgb/Camera_1/')
scene2_fog_bbox = pd.read_csv('./VKITTI_txt/Scene02/fog/bbox.txt', delimiter=' ')
scene2_fog_info = pd.read_csv('./VKITTI_txt/Scene02/fog/info.txt', delimiter=' ')
scene2_fog_pose = pd.read_csv('./VKITTI_txt/Scene02/fog/pose.txt', delimiter=' ')

scene2_morning = os.listdir('./VKITTI/Scene02/morning/frames/rgb/Camera_1/')
scene2_morning_bbox = pd.read_csv('./VKITTI_txt/Scene02/morning/bbox.txt', delimiter=' ')
scene2_morning_info = pd.read_csv('./VKITTI_txt/Scene02/morning/info.txt', delimiter=' ')
scene2_morning_pose = pd.read_csv('./VKITTI_txt/Scene02/morning/pose.txt', delimiter=' ')

scene2_overcast = os.listdir('./VKITTI/Scene02/overcast/frames/rgb/Camera_1/')
scene2_overcast_bbox = pd.read_csv('./VKITTI_txt/Scene02/overcast/bbox.txt', delimiter=' ')
scene2_overcast_info = pd.read_csv('./VKITTI_txt/Scene02/overcast/info.txt', delimiter=' ')
scene2_overcast_pose = pd.read_csv('./VKITTI_txt/Scene02/overcast/pose.txt', delimiter=' ')

scene2_rain = os.listdir('./VKITTI/Scene02/rain/frames/rgb/Camera_1/')
scene2_rain_bbox = pd.read_csv('./VKITTI_txt/Scene02/rain/bbox.txt', delimiter=' ')
scene2_rain_info = pd.read_csv('./VKITTI_txt/Scene02/rain/info.txt', delimiter=' ')
scene2_rain_pose = pd.read_csv('./VKITTI_txt/Scene02/rain/pose.txt', delimiter=' ')

scene2_sunset = os.listdir('./VKITTI/Scene02/sunset/frames/rgb/Camera_1/')
scene2_sunset_bbox = pd.read_csv('./VKITTI_txt/Scene02/sunset/bbox.txt', delimiter=' ')
scene2_sunset_info = pd.read_csv('./VKITTI_txt/Scene02/sunset/info.txt', delimiter=' ')
scene2_sunset_pose = pd.read_csv('./VKITTI_txt/Scene02/sunset/pose.txt', delimiter=' ')

# scene 6
scene6_clone = os.listdir('./VKITTI/Scene06/clone/frames/rgb/Camera_1/')
scene6_clone_bbox = pd.read_csv('./VKITTI_txt/Scene06/clone/bbox.txt', delimiter=' ')
scene6_clone_info = pd.read_csv('./VKITTI_txt/Scene06/clone/info.txt', delimiter=' ')
scene6_clone_pose = pd.read_csv('./VKITTI_txt/Scene06/clone/pose.txt', delimiter=' ')

scene6_fog = os.listdir('./VKITTI/Scene06/fog/frames/rgb/Camera_1/')
scene6_fog_bbox = pd.read_csv('./VKITTI_txt/Scene06/fog/bbox.txt', delimiter=' ')
scene6_fog_info = pd.read_csv('./VKITTI_txt/Scene06/fog/info.txt', delimiter=' ')
scene6_fog_pose = pd.read_csv('./VKITTI_txt/Scene06/fog/pose.txt', delimiter=' ')

scene6_morning = os.listdir('./VKITTI/Scene06/morning/frames/rgb/Camera_1/')
scene6_morning_bbox = pd.read_csv('./VKITTI_txt/Scene06/morning/bbox.txt', delimiter=' ')
scene6_morning_info = pd.read_csv('./VKITTI_txt/Scene06/morning/info.txt', delimiter=' ')
scene6_morning_pose = pd.read_csv('./VKITTI_txt/Scene06/morning/pose.txt', delimiter=' ')

scene6_overcast = os.listdir('./VKITTI/Scene06/overcast/frames/rgb/Camera_1/')
scene6_overcast_bbox = pd.read_csv('./VKITTI_txt/Scene06/overcast/bbox.txt', delimiter=' ')
scene6_overcast_info = pd.read_csv('./VKITTI_txt/Scene06/overcast/info.txt', delimiter=' ')
scene6_overcast_pose = pd.read_csv('./VKITTI_txt/Scene06/overcast/pose.txt', delimiter=' ')

scene6_rain = os.listdir('./VKITTI/Scene06/rain/frames/rgb/Camera_1/')
scene6_rain_bbox = pd.read_csv('./VKITTI_txt/Scene06/rain/bbox.txt', delimiter=' ')
scene6_rain_info = pd.read_csv('./VKITTI_txt/Scene06/rain/info.txt', delimiter=' ')
scene6_rain_pose = pd.read_csv('./VKITTI_txt/Scene06/rain/pose.txt', delimiter=' ')

scene6_sunset = os.listdir('./VKITTI/Scene06/sunset/frames/rgb/Camera_1/')
scene6_sunset_bbox = pd.read_csv('./VKITTI_txt/Scene06/sunset/bbox.txt', delimiter=' ')
scene6_sunset_info = pd.read_csv('./VKITTI_txt/Scene06/sunset/info.txt', delimiter=' ')
scene6_sunset_pose = pd.read_csv('./VKITTI_txt/Scene06/sunset/pose.txt', delimiter=' ')

# scene 18
scene18_clone = os.listdir('./VKITTI/Scene18/clone/frames/rgb/Camera_1/')
scene18_clone_bbox = pd.read_csv('./VKITTI_txt/Scene18/clone/bbox.txt', delimiter=' ')
scene18_clone_info = pd.read_csv('./VKITTI_txt/Scene18/clone/info.txt', delimiter=' ')
scene18_clone_pose = pd.read_csv('./VKITTI_txt/Scene18/clone/pose.txt', delimiter=' ')

scene18_fog = os.listdir('./VKITTI/Scene18/fog/frames/rgb/Camera_1/')
scene18_fog_bbox = pd.read_csv('./VKITTI_txt/Scene18/fog/bbox.txt', delimiter=' ')
scene18_fog_info = pd.read_csv('./VKITTI_txt/Scene18/fog/info.txt', delimiter=' ')
scene18_fog_pose = pd.read_csv('./VKITTI_txt/Scene18/fog/pose.txt', delimiter=' ')

scene18_morning = os.listdir('./VKITTI/Scene18/morning/frames/rgb/Camera_1/')
scene18_morning_bbox = pd.read_csv('./VKITTI_txt/Scene18/morning/bbox.txt', delimiter=' ')
scene18_morning_info = pd.read_csv('./VKITTI_txt/Scene18/morning/info.txt', delimiter=' ')
scene18_morning_pose = pd.read_csv('./VKITTI_txt/Scene18/morning/pose.txt', delimiter=' ')

scene18_overcast = os.listdir('./VKITTI/Scene18/overcast/frames/rgb/Camera_1/')
scene18_overcast_bbox = pd.read_csv('./VKITTI_txt/Scene18/overcast/bbox.txt', delimiter=' ')
scene18_overcast_info = pd.read_csv('./VKITTI_txt/Scene18/overcast/info.txt', delimiter=' ')
scene18_overcast_pose = pd.read_csv('./VKITTI_txt/Scene18/overcast/pose.txt', delimiter=' ')

scene18_rain = os.listdir('./VKITTI/Scene18/rain/frames/rgb/Camera_1/')
scene18_rain_bbox = pd.read_csv('./VKITTI_txt/Scene18/rain/bbox.txt', delimiter=' ')
scene18_rain_info = pd.read_csv('./VKITTI_txt/Scene18/rain/info.txt', delimiter=' ')
scene18_rain_pose = pd.read_csv('./VKITTI_txt/Scene18/rain/pose.txt', delimiter=' ')

scene18_sunset = os.listdir('./VKITTI/Scene18/sunset/frames/rgb/Camera_1/')
scene18_sunset_bbox = pd.read_csv('./VKITTI_txt/Scene18/sunset/bbox.txt', delimiter=' ')
scene18_sunset_info = pd.read_csv('./VKITTI_txt/Scene18/sunset/info.txt', delimiter=' ')
scene18_sunset_pose = pd.read_csv('./VKITTI_txt/Scene18/sunset/pose.txt', delimiter=' ')

# scene 20
scene20_clone = os.listdir('./VKITTI/Scene20/clone/frames/rgb/Camera_1/')
scene20_clone_bbox = pd.read_csv('./VKITTI_txt/Scene20/clone/bbox.txt', delimiter=' ')
scene20_clone_info = pd.read_csv('./VKITTI_txt/Scene20/clone/info.txt', delimiter=' ')
scene20_clone_pose = pd.read_csv('./VKITTI_txt/Scene20/clone/pose.txt', delimiter=' ')

scene20_fog = os.listdir('./VKITTI/Scene20/fog/frames/rgb/Camera_1/')
scene20_fog_bbox = pd.read_csv('./VKITTI_txt/Scene20/fog/bbox.txt', delimiter=' ')
scene20_fog_info = pd.read_csv('./VKITTI_txt/Scene20/fog/info.txt', delimiter=' ')
scene20_fog_pose = pd.read_csv('./VKITTI_txt/Scene20/fog/pose.txt', delimiter=' ')

scene20_morning = os.listdir('./VKITTI/Scene20/morning/frames/rgb/Camera_1/')
scene20_morning_bbox = pd.read_csv('./VKITTI_txt/Scene20/morning/bbox.txt', delimiter=' ')
scene20_morning_info = pd.read_csv('./VKITTI_txt/Scene20/morning/info.txt', delimiter=' ')
scene20_morning_pose = pd.read_csv('./VKITTI_txt/Scene20/morning/pose.txt', delimiter=' ')

scene20_overcast = os.listdir('./VKITTI/Scene20/overcast/frames/rgb/Camera_1/')
scene20_overcast_bbox = pd.read_csv('./VKITTI_txt/Scene20/overcast/bbox.txt', delimiter=' ')
scene20_overcast_info = pd.read_csv('./VKITTI_txt/Scene02/overcast/info.txt', delimiter=' ')
scene20_overcast_pose = pd.read_csv('./VKITTI_txt/Scene20/overcast/pose.txt', delimiter=' ')

scene20_rain = os.listdir('./VKITTI/Scene20/rain/frames/rgb/Camera_1/')
scene20_rain_bbox = pd.read_csv('./VKITTI_txt/Scene20/rain/bbox.txt', delimiter=' ')
scene20_rain_info = pd.read_csv('./VKITTI_txt/Scene20/rain/info.txt', delimiter=' ')
scene20_rain_pose = pd.read_csv('./VKITTI_txt/Scene20/rain/pose.txt', delimiter=' ')

scene20_sunset = os.listdir('./VKITTI/Scene20/sunset/frames/rgb/Camera_1/')
scene20_sunset_bbox = pd.read_csv('./VKITTI_txt/Scene20/sunset/bbox.txt', delimiter=' ')
scene20_sunset_info = pd.read_csv('./VKITTI_txt/Scene20/sunset/info.txt', delimiter=' ')
scene20_sunset_pose = pd.read_csv('./VKITTI_txt/Scene20/sunset/pose.txt', delimiter=' ')


In [17]:
# scene1 Preprocessing
length = len(scene1_clone) // 6

scene1_vkitti_data = pd.DataFrame()

for i in range(6):
    # 데이터 feature 추출
    # 'clone'
    if i == 0:
    
        # bbox 데이터에서 xmin, ymin, xmax, ymax value 뽑아내기
        # camera_ID == 1만 사용
        scene1_clone_bbox.rename(columns={'left':'xmin', 'right':'xmax', 'top':'ymin', 'bottom':'ymax'}, inplace=True)
        scene1_clone_bbox = scene1_clone_bbox[scene1_clone_bbox['cameraID']==1][['xmin','ymin','xmax','ymax','trackID','frame']]

        filename = [scene1_clone[fr] for fr in scene1_clone_bbox['frame'].values]
        scene1_clone_bbox.drop('frame', axis=1, inplace=True)

        # trackID에 맞는 label을 대응
        label_list = [scene1_clone_info[scene1_clone_info['trackID']==trackID]['label'].values[0] \
                      for trackID in scene1_clone_bbox['trackID'].values] 
        scene1_clone_bbox.drop('trackID', axis=1, inplace=True)

        scene1_clone_pose.rename(columns={'camera_space_Z':'zloc', 'alpha':'angle'}, inplace=True)
        scene1_clone_pose = scene1_clone_pose[scene1_clone_pose['cameraID']==1][['angle','zloc']]

        scene1_clone_bbox.reset_index(inplace=True)
        scene1_clone_pose.reset_index(inplace=True)
        scene1_clone_bbox.drop('index', axis=1, inplace=True)
        scene1_clone_pose.drop('index', axis=1, inplace=True)

        # 데이터 병합하기
        scene1_clone_data = pd.DataFrame({'filename':filename})
        scene1_clone_data['class'] = label_list
        scene1_clone_data = pd.concat([scene1_clone_data, scene1_clone_bbox], axis=1)
        scene1_clone_data = pd.concat([scene1_clone_data, scene1_clone_pose], axis=1)
        scene1_clone_data['weather'] = 'clone'
        scene1_clone_data['class'].replace({'Car':'car', 'Van':'car', 'Truck':'truck'}, inplace=True)

        set_filename = scene1_clone[length*i:length*(i+1)]
        mask = [index for index in range(len(scene1_clone_data)) if scene1_clone_data['filename'].values[index] in set_filename]
        scene1_clone_data = scene1_clone_data.iloc[mask]

        # 데이터 최종 병합
        scene1_vkitti_data = pd.concat([scene1_vkitti_data, scene1_clone_data], axis=0)
        
    # 'fog'
    elif i == 1:
        # bbox 데이터에서 xmin, ymin, xmax, ymax value 뽑아내기
        # camera_ID == 1만 사용
        scene1_fog_bbox.rename(columns={'left':'xmin', 'right':'xmax', 'top':'ymin', 'bottom':'ymax'}, inplace=True)
        scene1_fog_bbox = scene1_fog_bbox[scene1_fog_bbox['cameraID']==1][['xmin','ymin','xmax','ymax','trackID','frame']]

        filename = [scene1_fog[fr] for fr in scene1_fog_bbox['frame'].values]
        scene1_fog_bbox.drop('frame', axis=1, inplace=True)

        # trackID에 맞는 label을 대응
        label_list = [scene1_fog_info[scene1_fog_info['trackID']==trackID]['label'].values[0] \
                      for trackID in scene1_fog_bbox['trackID'].values] 
        scene1_fog_bbox.drop('trackID', axis=1, inplace=True)

        scene1_fog_pose.rename(columns={'camera_space_Z':'zloc', 'alpha':'angle'}, inplace=True)
        scene1_fog_pose = scene1_fog_pose[scene1_fog_pose['cameraID']==1][['angle','zloc']]

        scene1_fog_bbox.reset_index(inplace=True)
        scene1_fog_pose.reset_index(inplace=True)
        scene1_fog_bbox.drop('index', axis=1, inplace=True)
        scene1_fog_pose.drop('index', axis=1, inplace=True)

        # 데이터 병합하기
        scene1_fog_data = pd.DataFrame({'filename':filename})
        scene1_fog_data['class'] = label_list
        scene1_fog_data = pd.concat([scene1_fog_data, scene1_fog_bbox], axis=1)
        scene1_fog_data = pd.concat([scene1_fog_data, scene1_fog_pose], axis=1)
        scene1_fog_data['weather'] = 'fog'
        scene1_fog_data['class'].replace({'Car':'car', 'Van':'car', 'Truck':'truck'}, inplace=True)

        set_filename = scene1_fog[length*i:length*(i+1)]
        mask = [index for index in range(len(scene1_fog_data)) if scene1_fog_data['filename'].values[index] in set_filename]
        scene1_fog_data = scene1_fog_data.iloc[mask]

        # 데이터 최종 병합
        scene1_vkitti_data = pd.concat([scene1_vkitti_data, scene1_fog_data], axis=0)
        
    # 'morning'
    elif i == 2:
        # bbox 데이터에서 xmin, ymin, xmax, ymax value 뽑아내기
        # camera_ID == 1만 사용
        scene1_morning_bbox.rename(columns={'left':'xmin', 'right':'xmax', 'top':'ymin', 'bottom':'ymax'}, inplace=True)
        scene1_morning_bbox = scene1_morning_bbox[scene1_morning_bbox['cameraID']==1][['xmin','ymin','xmax','ymax','trackID','frame']]

        filename = [scene1_morning[fr] for fr in scene1_morning_bbox['frame'].values]
        scene1_morning_bbox.drop('frame', axis=1, inplace=True)

        # trackID에 맞는 label을 대응
        label_list = [scene1_morning_info[scene1_morning_info['trackID']==trackID]['label'].values[0] \
                      for trackID in scene1_morning_bbox['trackID'].values] 
        scene1_morning_bbox.drop('trackID', axis=1, inplace=True)

        scene1_morning_pose.rename(columns={'camera_space_Z':'zloc', 'alpha':'angle'}, inplace=True)
        scene1_morning_pose = scene1_morning_pose[scene1_morning_pose['cameraID']==1][['angle','zloc']]

        scene1_morning_bbox.reset_index(inplace=True)
        scene1_morning_pose.reset_index(inplace=True)
        scene1_morning_bbox.drop('index', axis=1, inplace=True)
        scene1_morning_pose.drop('index', axis=1, inplace=True)

        # 데이터 병합하기
        scene1_morning_data = pd.DataFrame({'filename':filename})
        scene1_morning_data['class'] = label_list
        scene1_morning_data = pd.concat([scene1_morning_data, scene1_morning_bbox], axis=1)
        scene1_morning_data = pd.concat([scene1_morning_data, scene1_morning_pose], axis=1)
        scene1_morning_data['weather'] = 'morning'
        scene1_morning_data['class'].replace({'Car':'car', 'Van':'car', 'Truck':'truck'}, inplace=True)

        set_filename = scene1_morning[length*i:length*(i+1)]
        mask = [index for index in range(len(scene1_morning_data)) if scene1_morning_data['filename'].values[index] in set_filename]
        scene1_morning_data = scene1_morning_data.iloc[mask]

        # 데이터 최종 병합
        scene1_vkitti_data = pd.concat([scene1_vkitti_data, scene1_morning_data], axis=0)
        
    # 'overcast'
    elif i == 3:
        # bbox 데이터에서 xmin, ymin, xmax, ymax value 뽑아내기
        # camera_ID == 1만 사용
        scene1_overcast_bbox.rename(columns={'left':'xmin', 'right':'xmax', 'top':'ymin', 'bottom':'ymax'}, inplace=True)
        scene1_overcast_bbox = scene1_overcast_bbox[scene1_overcast_bbox['cameraID']==1][['xmin','ymin','xmax','ymax','trackID','frame']]

        filename = [scene1_overcast[fr] for fr in scene1_overcast_bbox['frame'].values]
        scene1_overcast_bbox.drop('frame', axis=1, inplace=True)

        # trackID에 맞는 label을 대응
        label_list = [scene1_overcast_info[scene1_overcast_info['trackID']==trackID]['label'].values[0] \
                      for trackID in scene1_overcast_bbox['trackID'].values] 
        scene1_overcast_bbox.drop('trackID', axis=1, inplace=True)

        scene1_overcast_pose.rename(columns={'camera_space_Z':'zloc', 'alpha':'angle'}, inplace=True)
        scene1_overcast_pose = scene1_overcast_pose[scene1_overcast_pose['cameraID']==1][['angle','zloc']]

        scene1_overcast_bbox.reset_index(inplace=True)
        scene1_overcast_pose.reset_index(inplace=True)
        scene1_overcast_bbox.drop('index', axis=1, inplace=True)
        scene1_overcast_pose.drop('index', axis=1, inplace=True)

        # 데이터 병합하기
        scene1_overcast_data = pd.DataFrame({'filename':filename})
        scene1_overcast_data['class'] = label_list
        scene1_overcast_data = pd.concat([scene1_overcast_data, scene1_overcast_bbox], axis=1)
        scene1_overcast_data = pd.concat([scene1_overcast_data, scene1_overcast_pose], axis=1)
        scene1_overcast_data['weather'] = 'overcast'
        scene1_overcast_data['class'].replace({'Car':'car', 'Van':'car', 'Truck':'truck'}, inplace=True)

        set_filename = scene1_overcast[length*i:length*(i+1)]
        mask = [index for index in range(len(scene1_overcast_data)) if scene1_overcast_data['filename'].values[index] in set_filename]
        scene1_overcast_data = scene1_overcast_data.iloc[mask]

        # 데이터 최종 병합
        scene1_vkitti_data = pd.concat([scene1_vkitti_data, scene1_overcast_data], axis=0)
        
    # 'rain'
    elif i == 4:
        # bbox 데이터에서 xmin, ymin, xmax, ymax value 뽑아내기
        # camera_ID == 1만 사용
        scene1_rain_bbox.rename(columns={'left':'xmin', 'right':'xmax', 'top':'ymin', 'bottom':'ymax'}, inplace=True)
        scene1_rain_bbox = scene1_rain_bbox[scene1_rain_bbox['cameraID']==1][['xmin','ymin','xmax','ymax','trackID','frame']]

        filename = [scene1_rain[fr] for fr in scene1_rain_bbox['frame'].values]
        scene1_rain_bbox.drop('frame', axis=1, inplace=True)

        # trackID에 맞는 label을 대응
        label_list = [scene1_rain_info[scene1_rain_info['trackID']==trackID]['label'].values[0] \
                      for trackID in scene1_rain_bbox['trackID'].values] 
        scene1_rain_bbox.drop('trackID', axis=1, inplace=True)

        scene1_rain_pose.rename(columns={'camera_space_Z':'zloc', 'alpha':'angle'}, inplace=True)
        scene1_rain_pose = scene1_rain_pose[scene1_rain_pose['cameraID']==1][['angle','zloc']]

        scene1_rain_bbox.reset_index(inplace=True)
        scene1_rain_pose.reset_index(inplace=True)
        scene1_rain_bbox.drop('index', axis=1, inplace=True)
        scene1_rain_pose.drop('index', axis=1, inplace=True)

        # 데이터 병합하기
        scene1_rain_data = pd.DataFrame({'filename':filename})
        scene1_rain_data['class'] = label_list
        scene1_rain_data = pd.concat([scene1_rain_data, scene1_rain_bbox], axis=1)
        scene1_rain_data = pd.concat([scene1_rain_data, scene1_rain_pose], axis=1)
        scene1_rain_data['weather'] = 'rain'
        scene1_rain_data['class'].replace({'Car':'car', 'Van':'car', 'Truck':'truck'}, inplace=True)

        set_filename = scene1_rain[length*i:length*(i+1)]
        mask = [index for index in range(len(scene1_rain_data)) if scene1_rain_data['filename'].values[index] in set_filename]
        scene1_rain_data = scene1_rain_data.iloc[mask]

        # 데이터 최종 병합
        scene1_vkitti_data = pd.concat([scene1_vkitti_data, scene1_rain_data], axis=0)
        
    # 'sunset'
    elif i == 5:
        # bbox 데이터에서 xmin, ymin, xmax, ymax value 뽑아내기
        # camera_ID == 1만 사용
        scene1_sunset_bbox.rename(columns={'left':'xmin', 'right':'xmax', 'top':'ymin', 'bottom':'ymax'}, inplace=True)
        scene1_sunset_bbox = scene1_sunset_bbox[scene1_sunset_bbox['cameraID']==1][['xmin','ymin','xmax','ymax','trackID','frame']]

        filename = [scene1_sunset[fr] for fr in scene1_sunset_bbox['frame'].values]
        scene1_sunset_bbox.drop('frame', axis=1, inplace=True)

        # trackID에 맞는 label을 대응
        label_list = [scene1_sunset_info[scene1_sunset_info['trackID']==trackID]['label'].values[0] \
                      for trackID in scene1_sunset_bbox['trackID'].values] 
        scene1_sunset_bbox.drop('trackID', axis=1, inplace=True)

        scene1_sunset_pose.rename(columns={'camera_space_Z':'zloc', 'alpha':'angle'}, inplace=True)
        scene1_sunset_pose = scene1_sunset_pose[scene1_sunset_pose['cameraID']==1][['angle','zloc']]

        scene1_sunset_bbox.reset_index(inplace=True)
        scene1_sunset_pose.reset_index(inplace=True)
        scene1_sunset_bbox.drop('index', axis=1, inplace=True)
        scene1_sunset_pose.drop('index', axis=1, inplace=True)
        print(scene1_sunset_bbox)

        # 데이터 병합하기
        scene1_sunset_data = pd.DataFrame({'filename':filename})
        scene1_sunset_data['class'] = label_list
        scene1_sunset_data = pd.concat([scene1_sunset_data, scene1_sunset_bbox], axis=1)
        scene1_sunset_data = pd.concat([scene1_sunset_data, scene1_sunset_pose], axis=1)
        scene1_sunset_data['weather'] = 'sunset'
        scene1_sunset_data['class'].replace({'Car':'car', 'Van':'car', 'Truck':'truck'}, inplace=True)

        set_filename = scene1_sunset[length*i:length*(i+1)]
        mask = [index for index in range(len(scene1_sunset_data)) if scene1_sunset_data['filename'].values[index] in set_filename]
        scene1_sunset_data = scene1_sunset_data.iloc[mask]

        # 데이터 최종 병합
        scene1_vkitti_data = pd.concat([scene1_vkitti_data, scene1_sunset_data], axis=0)
        

      xmin  ymin  xmax  ymax
0      730   167  1207   374
1      696   186   832   281
2      677   185   749   246
3      377   201   459   256
4      500   195   534   220
...    ...   ...   ...   ...
6948     0   208   252   374
6949     0   209   225   374
6950     0   212   194   374
6951     0   214   156   374
6952     0   218   111   374

[6953 rows x 4 columns]


In [18]:
scene1_vkitti_data

,filename,class,xmin,ymin,xmax,ymax,angle,zloc,weather
0,rgb_00000.jpg,car,730,167,1207,374,-1.925707,6.406569,clone
1,rgb_00000.jpg,car,696,186,832,281,-1.757717,13.158640,clone
2,rgb_00000.jpg,car,677,185,749,246,-1.626971,19.291350,clone
3,rgb_00000.jpg,car,377,201,459,256,1.853136,23.765340,clone
4,rgb_00000.jpg,car,500,195,534,220,1.700254,46.440570,clone
...,...,...,...,...,...,...,...,...,...
6948,rgb_00421.jpg,car,0,208,252,374,2.164934,7.935210,sunset
6949,rgb_00422.jpg,car,0,209,225,374,2.199708,7.163083,sunset
6950,rgb_00423.jpg,car,0,212,194,374,2.240986,6.390986,sunset
6951,rgb_00424.jpg,car,0,214,156,374,2.289963,5.618877,sunset


In [30]:
scene1_vkitti_data.isnull().sum(axis=0)

filename    0
class       0
xmin        0
ymin        0
xmax        0
ymax        0
angle       0
zloc        0
weather     0
dtype: int64

In [20]:
scene1_vkitti_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6953 entries, 0 to 6952
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  6953 non-null   object 
 1   class     6953 non-null   object 
 2   xmin      6953 non-null   int64  
 3   ymin      6953 non-null   int64  
 4   xmax      6953 non-null   int64  
 5   ymax      6953 non-null   int64  
 6   angle     6953 non-null   float64
 7   zloc      6953 non-null   float64
 8   weather   6953 non-null   object 
dtypes: float64(2), int64(4), object(3)
memory usage: 543.2+ KB


In [31]:
#scene1_vkitti_data.to_csv('./scene1_vkitti_data.csv', index=False)